In [43]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from io import StringIO
from concurrent.futures import ThreadPoolExecutor, as_completed
from lxml import etree
import time
import numpy as np
import pandas as pd
import re
import requests

In [4]:
def scrape_table(table):
    data = []
    for row in table.find_all("tr")[1:]:  # Skip the header row
        cells = row.find_all("td")
        row_data = [cell.text.strip() for cell in cells]
        data.append(row_data)
    return data

In [ ]:
### Code for test scraping with S&P500 ###
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)
driver = webdriver.Chrome(options=chrome_options)
driver.get("https://stockanalysis.com/list/sp-500-stocks/")

table = driver.find_element("id", "main-table").get_attribute("outerHTML")
driver.quit()

df = pd.read_html(StringIO(table))[0]
df.drop("No.", inplace=True, axis=1)

In [52]:
k = 30    # Top k stocks by market cap
top_k_stocks_by_mkt_cap = list(df.iloc[:k]["Symbol"])

In [53]:
chrome_options = Options()
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--headless")
chrome_options.add_argument(
    "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36")
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("useAutomationExtension", False)

driver = webdriver.Chrome(options=chrome_options)

scrape_results_list = []
web_driver_wait_max_time = 2   # Duration for each scraping wait

for stock_ticker in top_k_stocks_by_mkt_cap:
    stock_ticker = stock_ticker.replace(".", "-")
    chunk_collector = {}
    try:
        driver.get(f"https://www.stocktargetadvisor.com/stock/USA/NYSE/{stock_ticker}")
        avg_analyst_rating = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[1]/div[2]/div[3]/div[1]/div[1]/div/figcaption/p"))
).text
        stock_target_advisor_analysis = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[1]/div[2]/div[3]/div[1]/div[3]/div/figcaption/p"))
).text
        avg_user_rating = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[1]/div[2]/div[3]/div[1]/div[4]/div/figcaption/p"))
).text
        avg_analyst_target = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[2]/div[1]/div[1]/div[3]/div/div[1]/div[1]"))
).text
        avg_upside_potential = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[2]/div[1]/div[1]/div[3]/div/div[1]/div[2]/span"))
).text
        chunk_collector["stock_ticker"] = stock_ticker
        chunk_collector["avg_analyst_rating"] = avg_analyst_rating
        chunk_collector["stock_target_advisor_analysis"] = stock_target_advisor_analysis
        chunk_collector["avg_user_rating"] = avg_user_rating
        chunk_collector["avg_analyst_target"] = avg_analyst_target
        chunk_collector["avg_upside_potential"] = avg_upside_potential
        scrape_results_list.append(chunk_collector)
    except:
        print(f"Error scraping for {stock_ticker}")
        continue

Error scraping for BRK-B


In [54]:
pd.DataFrame(scrape_results_list)

,stock_ticker,avg_analyst_rating,stock_target_advisor_analysis,avg_user_rating,avg_analyst_target,avg_upside_potential
0,MSFT,Strong Buy,Slightly Bullish,Strong Buy,USD 504.80,(+11.54%)
1,NVDA,Strong Buy,Slightly Bullish,Hold,USD 169.67,(+28.74%)
2,AAPL,Buy,Bullish,Buy,USD 246.58,(+22.01%)
3,AMZN,Strong Buy,Slightly Bullish,Buy,USD 253.59,(+26.09%)
4,GOOG,Strong Buy,Slightly Bullish,Buy,USD 209.43,(+23.15%)
5,GOOGL,Strong Buy,Bullish,Strong Buy,USD 206.69,(+22.62%)
6,META,Strong Buy,Neutral,Buy,USD 713.45,(+12.27%)
7,TSLA,Buy,Slightly Bullish,Buy,USD 335.59,(+0.29%)
8,AVGO,Strong Buy,Slightly Bullish,Buy,USD 235.86,(+2.67%)
9,WMT,Strong Buy,Slightly Bullish,Buy,USD 105.52,(+9.43%)


In [55]:
def scrape_stock(stock_ticker):
    chrome_options = Options()
    chrome_options.add_argument("--disable-blink-features=AutomationControlled")
    chrome_options.add_argument("--headless")
    chrome_options.add_argument(
        "user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/93.0.4577.82 Safari/537.36")
    chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
    chrome_options.add_experimental_option("useAutomationExtension", False)
    driver = webdriver.Chrome(options=chrome_options)
    
    web_driver_wait_max_time = 2   # Duration for each scraping wait
    
    stock_ticker = stock_ticker.replace(".", "-")
    chunk_collector = {}
    try:
        driver.get(f"https://www.stocktargetadvisor.com/stock/USA/NYSE/{stock_ticker}")
        avg_analyst_rating = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[1]/div[2]/div[3]/div[1]/div[1]/div/figcaption/p"))
).text
        stock_target_advisor_analysis = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[1]/div[2]/div[3]/div[1]/div[3]/div/figcaption/p"))
).text
        avg_user_rating = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[1]/div[2]/div[3]/div[1]/div[4]/div/figcaption/p"))
).text
        avg_analyst_target = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[2]/div[1]/div[1]/div[3]/div/div[1]/div[1]"))
).text
        avg_upside_potential = WebDriverWait(driver, web_driver_wait_max_time).until(
    EC.presence_of_element_located((By.XPATH, "/html/body/div[1]/div[1]/div/section/div[2]/div[2]/div[1]/div[1]/div[3]/div/div[1]/div[2]/span"))
).text
        chunk_collector["stock_ticker"] = stock_ticker
        chunk_collector["avg_analyst_rating"] = avg_analyst_rating
        chunk_collector["stock_target_advisor_analysis"] = stock_target_advisor_analysis
        chunk_collector["avg_user_rating"] = avg_user_rating
        chunk_collector["avg_analyst_target"] = avg_analyst_target
        chunk_collector["avg_upside_potential"] = avg_upside_potential
        scrape_results_list.append(chunk_collector)
        return chunk_collector
    except Exception as e:
        print(f"Error scraping for {stock_ticker}: {e}")
        return None
    finally:
        driver.quit()

In [56]:
max_workers = 5 # Number of concurrent threads

with ThreadPoolExecutor(max_workers=max_workers) as executor:  
    futures = [executor.submit(scrape_stock, ticker) for ticker in top_k_stocks_by_mkt_cap]
    results = [f.result() for f in as_completed(futures) if f.result() is not None]

Error scraping for BRK-B: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF69874CF45+75717]
	GetHandleVerifier [0x00007FF69874CFA0+75808]
	(No symbol) [0x00007FF698518F9A]
	(No symbol) [0x00007FF69856F4C6]
	(No symbol) [0x00007FF69856F77C]
	(No symbol) [0x00007FF6985C2577]
	(No symbol) [0x00007FF6985973BF]
	(No symbol) [0x00007FF6985BF39C]
	(No symbol) [0x00007FF698597153]
	(No symbol) [0x00007FF698560421]
	(No symbol) [0x00007FF6985611B3]
	GetHandleVerifier [0x00007FF698A4D71D+3223453]
	GetHandleVerifier [0x00007FF698A47CC2+3200322]
	GetHandleVerifier [0x00007FF698A65AF3+3322739]
	GetHandleVerifier [0x00007FF698766A1A+180890]
	GetHandleVerifier [0x00007FF69876E11F+211359]
	GetHandleVerifier [0x00007FF698755294+109332]
	GetHandleVerifier [0x00007FF698755442+109762]
	GetHandleVerifier [0x00007FF69873BA59+4825]
	BaseThreadInitThunk [0x00007FF973B2E8D7+23]
	RtlUserThreadStart [0x00007FF9754FC5DC+44]



In [57]:
pd.DataFrame(results)

,stock_ticker,avg_analyst_rating,stock_target_advisor_analysis,avg_user_rating,avg_analyst_target,avg_upside_potential
0,GOOG,Strong Buy,Slightly Bullish,Buy,USD 209.43,(+23.15%)
1,AMZN,Strong Buy,Slightly Bullish,Buy,USD 253.59,(+26.09%)
2,MSFT,Strong Buy,Slightly Bullish,Strong Buy,USD 504.80,(+11.54%)
3,AAPL,Buy,Bullish,Buy,USD 246.58,(+22.01%)
4,NVDA,Strong Buy,Slightly Bullish,Hold,USD 169.67,(+28.74%)
5,META,Strong Buy,Neutral,Buy,USD 713.45,(+12.27%)
6,GOOGL,Strong Buy,Bullish,Strong Buy,USD 206.69,(+22.62%)
7,TSLA,Buy,Slightly Bullish,Buy,USD 335.59,(+0.29%)
8,AVGO,Strong Buy,Slightly Bullish,Buy,USD 235.86,(+2.67%)
9,V,Strong Buy,Neutral,Buy,USD 366.03,(+2.16%)
